In [13]:
#!pip3 install --upgrade --quiet langchain langcahin-community langchain-openai chromadb
#!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv
!pip3 install chromadb

  Using cached chromadb-1.3.4-cp39-abi3-win_amd64.whl.metadata (7.4 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pybase64-1.4.2-cp312-cp312-win_amd64.whl.metadata (9.0 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.2-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.38.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl


ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'C:\\users\\boutc\\appdata\\local\\pip\\cache\\wheels\\d5\\3d\\69\\8d68d249cd3de2584f226e27fd431d6344f7d70fd856ebd01b\\pypika-0.48.9-py2.py3-none-any.whl'
Check the permissions.



In [1]:
# Import Langchain modules
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.utils.pydantic import BaseModel, Field

# Other modules and packages
import os
import tempfile
import streamlit as st  
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
#llm.invoke("Tell me a joke about cats")

In [5]:
loader = PyPDFLoader("data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf")
pages = loader.load()
pages

Ignoring wrong pointing object 23 0 (offset 0)


[Document(metadata={'producer': 'Mac OS X 10.13.5 Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20180624213431Z00'00'", 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'moddate': "D:20180624213431Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='APPLIED COGNITIVE PSYCHOLOGY\nAppl. Cognit. Psychol. 20: 139–156 (2006)\nPublished online 31 October 2005 in Wiley InterScience\n(www.interscience.wiley.com) DOI: 10.1002/acp.1178\nConsequences of Erudite Vernacular Utilized Irrespective\nof Necessity: Problems with Using Long Words Needlessly\nDANIEL M. OPPENHEIMER*\nPrinceton University, USA\nSUMMARY\nMost texts on writing style encourage authors to avoid overly-complex words. However, a majority\nof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to give\nthe impression of intelligence. This pa

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)
chunks

[Document(metadata={'producer': 'Mac OS X 10.13.5 Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20180624213431Z00'00'", 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'moddate': "D:20180624213431Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='APPLIED COGNITIVE PSYCHOLOGY\nAppl. Cognit. Psychol. 20: 139–156 (2006)\nPublished online 31 October 2005 in Wiley InterScience\n(www.interscience.wiley.com) DOI: 10.1002/acp.1178\nConsequences of Erudite Vernacular Utilized Irrespective\nof Necessity: Problems with Using Long Words Needlessly\nDANIEL M. OPPENHEIMER*\nPrinceton University, USA\nSUMMARY\nMost texts on writing style encourage authors to avoid overly-complex words. However, a majority\nof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to give\nthe impression of intelligence. This pa

In [7]:
def get_embedding_function():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY
    )
    return embeddings

embedding_function = get_embedding_function()
#test_vector = embedding_function.embed_query("cat")

In [8]:
from langchain_classic.evaluation.loading import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance", 
                            embeddings=embedding_function)

#evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop")

In [9]:

#evaluator.evaluate_strings(prediction="Paris", reference="coffeeshop")

In [10]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()
    
    return vectorstore

In [11]:

# Create vectorstore
vectorstore = create_vectorstore(chunks=chunks, 
                                 embedding_function=embedding_function, 
                                 vectorstore_path="vectorstore_chroma")

C:\Users\boutc\AppData\Local\Temp\ipykernel_31188\2260934871.py:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [12]:
# Load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_chroma", embedding_function=embedding_function)

C:\Users\boutc\AppData\Local\Temp\ipykernel_31188\2183088283.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="vectorstore_chroma", embedding_function=embedding_function)


In [13]:
# Create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("What is the title of the paper?")
relevant_chunks

[Document(metadata={'creator': 'Preview', 'keywords': '', 'creationdate': "D:20180624213431Z00'00'", 'total_pages': 18, 'aapl:keywords': '[]', 'page': 14, 'page_label': '15', 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'producer': 'Mac OS X 10.13.5 Quartz PDFContext', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'moddate': "D:20180624213431Z00'00'"}, page_content='New York: Cambridge Press.\nKelley, H. H. (1973). The process of causal attribution. American Psychologist, 28 , 107–128.\nMacKinnon, D. P., Lockwood, C. M., Hoffman, J. M., West, S. G., & Sheets, V. (2002). A\ncomparison of methods to test mediation and other intervening variable effects. Psychological\nMethods, 7 (1), 83–104.\nNorwick, R. J., & Epley, N. (November, 2002). Conﬁdence as inference from subjective experience .\nTalk presented at the meeting of the Society for Judgment and Decision Making, Kansas City,\nMO.\nOppenheimer, D. M. (2004). Spontaneous discounting of availabili

In [14]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

In [15]:
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="What is the title of the paper?")
print(prompt)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

New York: Cambridge Press.
Kelley, H. H. (1973). The process of causal attribution. American Psychologist, 28 , 107–128.
MacKinnon, D. P., Lockwood, C. M., Hoffman, J. M., West, S. G., & Sheets, V. (2002). A
comparison of methods to test mediation and other intervening variable effects. Psychological
Methods, 7 (1), 83–104.
Norwick, R. J., & Epley, N. (November, 2002). Conﬁdence as inference from subjective experience .
Talk presented at the meeting of the Society for Judgment and Decision Making, Kansas City,
MO.
Oppenheimer, D. M. (2004). Spontaneous discounting of availability in frequency judgment tasks.
Psychological Science, 15 (2), 100–105.
Problems with long words 153
Copyright # 2005 John Wiley & Sons, Ltd. Appl. Cognit. Psychol. 20: 139–156 (2006)

---

Oppenheimer, D. M

In [17]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
rag_chain.invoke("What's the title of this paper?")

AIMessage(content='The title of the paper is "A rose in any other font wouldn’t smell as sweet: fluency effects in categorization."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 1535, 'total_tokens': 1561, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYpScXTBgS1ntBkQ9L9f5cwzda9zw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--aae3b72f-b1ff-488b-b0f8-ebf2c1725872-0', usage_metadata={'input_tokens': 1535, 'output_tokens': 26, 'total_tokens': 1561, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:

class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources")
    
class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    paper_title: AnswerWithSources
    paper_summary: AnswerWithSources
    publication_year: AnswerWithSources
    paper_authors: AnswerWithSources

In [19]:
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.with_structured_output(ExtractedInfo, strict=True)
        )

rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")

ExtractedInfo(paper_title=AnswerWithSources(answer='A rose in any other font wouldn’t smell as sweet: fluency effects in categorization', sources='Oppenheimer, D. M., & Frank, M. C. (submitted). A rose in any other font wouldn’t smell as sweet: fluency effects in categorization.', reasoning='The title is taken directly from the citation provided in the context.'), paper_summary=AnswerWithSources(answer='This study examines how the fluency of categorization affects the judgment of items based on their presentation, aiming to explore cognitive processes involved in categorization.', sources='Oppenheimer, D. M., & Frank, M. C. (submitted). A rose in any other font wouldn’t smell as sweet: fluency effects in categorization.', reasoning='The summary is inferred from the title and the associated concepts of fluency effects in categorization, as discussed in multiple sources.'), publication_year=AnswerWithSources(answer='2006', sources='Copyright # 2005 John Wiley & Sons, Ltd. Appl. Cognit. P

In [20]:

structured_response = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
df = pd.DataFrame([structured_response.dict()])

# Transforming into a table with two rows: 'answer' and 'source'
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

# Create new dataframe with two rows: 'answer' and 'source'
structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer', 'source', 'reasoning'])
structured_response_df

C:\Users\boutc\AppData\Local\Temp\ipykernel_31188\3122578211.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  df = pd.DataFrame([structured_response.dict()])


,paper_title,paper_summary,publication_year,paper_authors
answer,A rose in any other font wouldn’t smell as swe...,This paper discusses the effects of fluency in...,N/A,D. M. Oppenheimer and M. C. Frank
source,"Oppenheimer, D. M., & Frank, M. C. (submitted)...","Oppenheimer, D. M., & Frank, M. C. (submitted)...","Oppenheimer, D. M., & Frank, M. C. (submitted)...","Oppenheimer, D. M., & Frank, M. C. (submitted)..."
reasoning,The title of the research paper is directly me...,The summary is derived from the title indicati...,"The paper is listed as 'submitted', meaning th...",The authors are clearly mentioned in the conte...
